# Scikit-learn Regression Basic Flow
작성자: 고정훈

Regression은 Y target이 실수(real number)일 때 X input으로부터 Y를 예측하는 과제를 말합니다. X Input은 많은 경우 숫자나 카테고리이며, 드물게는 이미지나 시퀀스 혹은 기타 형태일 수 있습니다. 

X Input이 숫자나 카테고리인 기본 Regression은 Scikit-learn으로 시작하시길 권해드립니다. 전처리부터 학습까지 (딥러닝을 제외한) 머신러닝에 필요한 거의 모든 것을 갖추었다고 해도 무방합니다. 사용법도 간단하고, 인터페이스도 매우 훌륭합니다. 

이 페이지에서는 가장 간단하고 기본적인 Regression Flow를 Scikit-learn으로 구현해보겠습니다. 

## Regression Flow

Regression은 아래와 같은 흐름(flow)을 대략적으로 따릅니다. 

* Preprocessing
    1. Data Munging
    2. One-hot-encoding
    3. Missing Data Processing
    4. Scaling
    5. Feature Selection
    6. Dimensionality Reduction
    
    
* Training
    1. Train/Test Set Split 
    2. Algorithm Selection
    3. Hyper-parameter Selection
    4. Training 
    5. Validation 

전처리(Preprocessing)에서 순서는 바뀔 수 있습니다. 위 전체 과정이 반복되며 최적의 성능을 내는 조건을 찾습니다. 

## 기본 흐름 Scikit-learn 구현
가장 기본적인 상황에서 Regression Flow를 Scikit-learn으로 구현해보겠습니다. 

### 필요한 함수, 클래스 불러오기

In [3]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

예제에서 필요한 클래스와 함수를 scikit-learn으로부터 불러옵니다. scikit-learn library는 python에서 `sklearn`을 이용해 불러옵니다.
- `load_boston`은 regression에서 가장 먼저 해보는 데이터 셋 중 하나입니다. 보스턴 지역의 집값을 여러 요소로부터 예측하는 데이터입니다. `
- train_test_split`은 학습 데이터와 성능 검증에 이용되는 test 데이터를 나눠주는 함수입니다.  
- `StandardScaler`는 scaling을 위한 클래스입니다. Regression에 사용하는 대부분의 알고리즘은 사전에 데이터 scaling이 필요합니다.  Feature들 간의 단위가 다른 상황에서 scaling을 하지 않으면 단위가 크게 변하는 feature에 편중되서 모델을 만들기 때문입니다. 
- `RandomForestRegressor`는 Regression 모델 중 하나입니다. Hyper-parameter tuning이 거의 필요없는 모델이기 때문에 데이터를 처음 받았을 때 가장 먼저 시도해보기에 좋은 Regression을 알고리즘입니다. 

### 데이터 불러오기

In [4]:
x, y = load_boston(return_X_y=True)
print('x.shape: ', x.shape)
print('y.shape: ', y.shape)

x.shape:  (506, 13)
y.shape:  (506,)


데이터를 불러왔습니다. scikit-learn의 기본 데이터형은 numpy array입니다. 별 말이 없으면 numpy array 형태로 입력하고 numpy array 형태
로 반환한다고 생각하시면 됩니다. numpy의 `shape` 명령어로 x, y가 어떻게 구성되어 있는지 확인합니다. 총 506개 샘플이 있고, feature 수는 총 13개네요. Y target은 집값 하나입니다. 

### Training/Test Set 나누기

In [10]:
y = np.reshape(y, (-1, 1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print('x_train.shape: ', x_train.shape)
print('x_test.shape: ', x_test.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

x_train.shape:  (404, 13)
x_test.shape:  (102, 13)
y_train.shape:  (404, 1)
y_test.shape:  (102, 1)


`np.reshape`을 통해 y의 차원을 바꿔주는 이유는 scaling을 할 때 에러를 없애기 위해서입니다. stable 버전 중 현 시점(2018.04)에서 최신 버전인 Scikit-learn 0.19에서는 numpy array 차원이 1일 경우 scaler 동작 시 에러가 납니다. 

`train_test_split`에 x와 y를 넣어 training set과 test set을 8:2 비율로 나누고 잘 나누어졌는지 shape을 확인했습니다. 

### Scaling

대부분의 머신러닝 알고리즘은 사전 scaling을 필요로 합니다. Linear regression 처럼 scaling이 필요가 없거나, random forest처럼 상대적으로 덜 필요한 알고리즘은 있지만 어떤 알고리즘을 사용하든 scaling을 해서 손해볼 일은 없습니다. 

Scaling을 해야 하는 이유는 feature들의 범위(range)가 각각 다르기 때문입니다. 어떤 feature는 대략 1~10 사이의 범위를 가지는데 반해 어떤 feature는 1~10000 사이를 오간다면 1~10000 사이 feature를 더 중요하게 받아들이는 문제가 있는 거죠. 실제로는 그렇지 않음에도 불구하구요. 변이(variation)가 큰 feature라도 y에 미치는 영향이 그에 비례한다는 보장은 없습니다. 이를 위해 모든 feature가 비슷한 범위를 오가도록 scaling을 합니다. 

Scaling 하는 방법에는 크게는 두 가지가 있습니다. 평균은 0, 표준편차는 1로 변환시키는 standard scaling과 min은 0 또는 -1, max는 1로 scaling 하는 min max scaling이 있습니다. 현 예제에서는 standard scaling을 사용했습니다. 

In [33]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
y_train_scaled = y_scaler.fit_transform(y_train)

'fit_transform'은 앞으로 자주 보시게 될 method 이름입니다. Scikit-learn에서 *fit*은 instance를 데이터에 적합시키다는 의미입니다. '적합'은 맥락에 따라 의미가 다릅니다. Scaler에서 쓰일 때는 scaling factor를 계산해서 instance에 저장한다는 의미입니다. Standard scaling을 하려면 평균과 표준편차를 계산해야겠죠. *fit*은 그것들을 계산하라는 의미입니다. *transform*은 데이터를 변환하라는 의미입니다. 앞서 *fit*으로 평균과 표준편차를 구했으니 각 데이터에서 평균을 빼고 표준편차로 나누는 작업을 하라는 게 *transform*입니다. 

*fit_transform*은 *fit*을 한 후 곧바로 *transform*을 하라는 명령어입니다. 

그럼 정말 원하는대로 scaling이 잘 되었는지 확인해보겠습니다.

In [34]:
float_formatter = lambda x: "%.1f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})  # 보기 쉽도록 출력 형식을 소수점 한 자리수로 제한합니다. 

print('x mean: {}'.format(np.mean(x_train, axis=0)))
print('x stdev: {}'.format(np.std(x_train, axis=0)))

print('x_train_scaled mean: {}'.format(np.mean(x_train_scaled, axis=0)))
print('x_train_sclaed stdev: {}'.format(np.std(x_train_scaled, axis=0)))

x mean: [3.9 11.6 11.0 0.1 0.6 6.3 68.6 3.8 9.5 406.3 18.4 354.4 12.7]
x stdev: [9.3 23.8 6.7 0.3 0.1 0.7 28.1 2.1 8.7 165.8 2.2 95.5 7.3]
x_train_scaled mean: [0.0 0.0 0.0 0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0]
x_train_sclaed stdev: [1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0]


제대로 된 것을 확인할 수 있습니다.

### 학습(Learning) 하기

드디어 학습을 시작해보겠습니다. 본 예제에서는 `RandomForestRegressor`를 사용했습니다. 

In [37]:
regressor = RandomForestRegressor(n_estimators=500)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

먼저 regressor를 정의합니다. regressor는 RandomForestRegressor 클래스로부터 생성한 instance입니다. 
`n_estimators`는 RandomForestRegressor의 hyper-parameter입니다. Random forest는 decision tree라는 머신러닝 기법의 ensemble로 구성됩니다. ensemble이란 여러 모델로부터 평균을 내는 것을 의미합니다. n_estimators는 몇 개의 ensemble을 만들어 평균할지 정하는 hyper-parameter죠. Default 값은 10인데, 10을 입력하면 성능이 좋지 않습니다. `n_estimators` 숫자는 크면 클수록 좋으나 어느 이상되면 saturation 되므로 여기서는 500을 사용했습니다. 

In [38]:
regressor.fit(x_train_scaled, y_train_scaled.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

앞서 설명드렸듯 `fit`은 데이터에 적합시킨다는 의미입니다. Regressor에서 적합은 학습한다는 의미입니다. x와 y를 넣어주고 fit을 하면 학이 수행됩니다. 매우 쉽죠? 

한가지 유의하셔야 할점은 `y_train_scaled`를 그대로 넣은 게 아니라 `ravel`이라는 method로 평탄화 한 것입니다. `ravel`은 numpy array를 1차원으로 바꿔주는 역할을 합니다. RandomForestRegressor에 y를 넣을 때는 1차원으로 들어가야 합니다. Scikit-learn에서 어떤 Regressor는y를 1차원으로 입력해야 하며 어떤 것은 2차원으로 입력해야 합니다. Error나 Warning 메시지를 읽고 판단하셔야 합니다. `ravel`을 안 시키고 넣으면 아래와 같은 Warning 메시지가 뜹니다.

In [39]:
regressor.fit(x_train_scaled, y_train_scaled)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

Warning 메시지가 뜨는 것이 성가시지 않는다면 그대로 입력하셔도 무방합니다. 다만 y의 데이터 형식이 강제로 변환된 채 들어간다는 사실은 알고 있으면 좋겠죠. 원본 y_train_scaled의 차원은 변하지 않습니다. 

### Validation

이제 학습이 제대로 되었는지 검증해보겠습니다. 정합성을 검증할 때는 학습에 사용하지 않은 데이터를 이용해야 합니다. 학습한 데이터에 대해서는 과적합(overfitting)이 될 수도 있기 때문이죠. 

In [44]:
x_test_scaled = x_scaler.transform(x_test)  # fit_transform이 아니라 transfrom 임에 주의하세요 
y_test_scaled = y_scaler.transform(y_test)  # fit_transform이 아니라 transform 임에 주의하세요 

score = regressor.score(x_test_scaled, y_test_scaled)
print('Test R2 score: {:.2f}'.format(score))

Test R2 score: 0.83


앞서 분리해놨던 test set을 scaling 합니다. 이때 주의해야 할 점은 앞서 train set을 fit했던 정보를 그대로 써야한다는 것입니다. 만약 test set에 대해서 새로 `fit` 혹은 `fit_transform`을 한다면 traning set과 다른 평균과 표준편차로 scaling이 되어 엉뚱한 예측을 하게 됩니다. 

scikit-learn regressor에서 `score` method는 $R^2$(coefficient of determination) 값을 출력합니다. 
Training set에 대해서도 $R^2$를 구해볼까요?

In [46]:
score = regressor.score(x_train_scaled, y_train_scaled)
print('Train R2 score: {:.2f}'.format(score))

Train R2 score: 0.98


일반적으로 training $R^2$ score는 test $R^2$ score보다 높습니다. 앞서 말했듯 우리에게 중요한 건 training set에 대한 성적이 아니라 test set에 대한 성적입니다. 

### Prediction

x, y가 함께 있는 데이터로 학습을 하고 x만 있는 새로운 데이터가 온다고 가정합시다. 애초에 학습을 시키는 목적은 y가 없는 데이터에 대해서 y를 예측하기 위해서죠. 
실제 학습한 모델로 예측하기 위해서는 `predict`를 사용하면 됩니다. y가 없는 데이터는 현재 따로 없으니 test set을 x만 있는 데이터라고 생각하고 다시 한 번 사용하겠습니다. 

In [48]:
y_pred_scaled = regressor.predict(x_test_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
print(y_pred)

[33.6 20.1 22.5 19.9 22.0 15.5 20.0 15.0 12.6 14.6 22.4 30.1 15.3 21.0 19.9
 8.6 28.6 31.3 17.5 18.8 30.9 29.8 15.0 24.2 22.1 19.6 25.9 25.6 15.0 15.1
 26.2 20.3 12.6 32.2 29.0 23.2 17.4 13.2 26.1 16.7 27.1 32.7 23.5 34.1 18.5
 15.1 39.6 16.0 25.1 28.8 22.9 30.1 27.8 20.7 20.9 19.0 19.6 20.2 47.8 20.1
 18.3 20.1 20.5 19.5 22.3 26.3 34.6 20.1 15.4 45.7 21.4 27.4 22.4 17.1 45.3
 32.8 14.8 17.4 33.3 12.6 24.2 15.7 19.4 16.6 14.8 19.4 18.0 18.5 19.9 26.3
 19.8 22.6 26.6 21.7 21.5 15.9 14.7 21.0 21.7 8.9 16.6 24.7]


`predict` method를 이용하여 새롭게 들어온 x 데이터에 대해서 y를 예측해봤습니다. 여기서 주의하실 점은 scaling을 되돌려야 한다는 것입니다. scaler의 `inverse_transform`은 scaling이 된 데이터를 원래의 scale로 되돌리는 method입니다. 

### 전체 코드

Regression 모델을 Scikit-learn으로 간단히 구현해보았습니다. 아래는 하나로 합친 코드입니다. 

In [ ]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# 데이터 불러오기
x, y = load_boston(return_X_y=True)
y = np.reshape(y, (-1, 1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Scaling
x_scaler = StandardScaler()
y_scaler = StandardScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
y_train_scaled = y_scaler.fit_transform(y_train)

# 학습
regressor = RandomForestRegressor(n_estimators=500)
regressor.fit(x_train_scaled, y_train_scaled.ravel())

# 검증
score = regressor.score(x_test_scaled, y_test_scaled)
print('Test R2 score: {:.2f}'.format(score))

# 예측
y_pred_scaled = regressor.predict(x_test_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
print(y_pred)